<a href="https://colab.research.google.com/github/ksushenc1a/-/blob/main/Bert_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импорт библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader

# Загрузка данных
train_data = pd.read_csv('train.csv')  # Замените на актуальный путь
test_data = pd.read_csv('test.csv')    # Замените на актуальный путь

# Инициализация токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Токенизация данных с использованием параллельной обработки
train_encodings = tokenizer(train_data['text'].tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_data['text'].tolist(), truncation=True, padding=True, return_tensors='pt')

# Преобразование в тензоры
train_labels = torch.tensor(train_data['is_joke'].tolist())
test_labels = torch.tensor(test_data['is_joke'].tolist())

# Создание Dataset
class JokeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}  # Убрали лишнее преобразование в тензор
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = JokeDataset(train_encodings, train_labels)
test_dataset = JokeDataset(test_encodings, test_labels)

# Инициализация модели
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Увеличен размер батча
    per_device_eval_batch_size=16,    # Увеличен размер батча
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    fp16=True,  # Включение смешанной точности
)

# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Обучение модели
trainer.train()

In [ ]:
# Предсказание на тестовых данных
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(axis=1)

# Создание таблицы с примерами, предсказаниями
results_df = pd.DataFrame({
    'text': test_data['text'],
    'true_label': test_data['is_joke'],
    'predicted_label': predicted_labels
})

# Сохранение результатов в CSV файл
results_df.to_csv('predictions.csv', index=False)


In [ ]:
# Оценка модели
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(test_labels.numpy(), predicted_labels))
